# 데이터 전처리

목표1 : 노원과 인접구 데이터만 갖도록 가공, 포함범위 : 노원 도봉 중랑 성북

목적데이터 : cctv, 안심귀가길 시설물, 보안등

목표2 : 순찰반경내 데이터만 자르는 함수 생성

라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd

상수 지정

In [2]:
gu_list=["노원구", "도봉구", "중랑구", "성북구"]
data_path = "데이터전처리/"
encoding = 'cp949'

### 서울시 cctv가공

In [3]:
seoul_cctv = pd.read_csv(data_path+"seoul_cctv.csv", encoding=encoding)
processed_cctv = pd.DataFrame(columns=["위도", "경도"], dtype=np.float64)
for i in gu_list:
    temp = seoul_cctv.loc[seoul_cctv["자치구"]==i,["위도", "경도"]]
    processed_cctv = pd.concat([processed_cctv, temp])
processed_cctv

,위도,경도
48581,37.6734,127.0590
48582,37.6734,127.0590
48583,37.6734,127.0590
48584,37.6729,127.0588
48585,37.6729,127.0588
...,...,...
26702,37.5985,127.0327
26703,37.5985,127.0327
26704,37.5881,127.0075
26705,37.6096,127.0572


In [4]:
processed_cctv.to_csv(data_path+"processed_cctv.csv", index=False,  encoding=encoding)

### 서울시 안심귀가길 시설물 가공

In [5]:
seoul_ansim = pd.read_csv(data_path+"seoul_ansim.csv", encoding=encoding)
processed_ansim = pd.DataFrame()
for i in gu_list:
    processed_ansim = pd.concat([ processed_ansim, seoul_ansim.loc[seoul_ansim["시군구명"] == "서울특별시 "+i, "포인트 wkt"] ])

def word_to_EW(word):
    temp = word.split("(")
    temp = temp[-1].split(" ")
    return temp[0]
def word_to_NS(word):
    temp = word.split("(")
    temp = temp[-1].split(" ")
    return temp[1][:-1]

processed_ansim["위도"] = processed_ansim[0].apply(word_to_NS)
processed_ansim["경도"] = processed_ansim[0].apply(word_to_EW)
processed_ansim = processed_ansim.drop(columns=[0]).reset_index(drop=True)
processed_ansim

,위도,경도
0,37.624871,127.050828
1,37.624608,127.050897
2,37.624236,127.051045
3,37.624017,127.051316
4,37.624017,127.051316
...,...,...
2033,37.60735199999999878,127.05262000000000455
2034,37.60735700000000037,127.0526660000000021
2035,37.60735700000000037,127.0526660000000021
2036,37.6074300000000008,127.05291400000000124


In [6]:
processed_ansim.to_csv(data_path+"processed_ansim.csv", index=False, encoding=encoding)

### 서울시 보안등 가공

하려했으나 중랑구, 성북구는 데이터가 없어 제외

In [7]:
nowon_light = pd.read_csv(data_path+"nowon_light.csv", encoding=encoding)
dobong_light = pd.read_csv(data_path+"dobong_light.csv", encoding=encoding)
processed_light = pd.concat([nowon_light, dobong_light]).reset_index(drop=True)

In [8]:
processed_light.to_csv(data_path+"processed_light.csv", index=False,  encoding=encoding)

### 서울시 유흥주점 가공

범위내 영업중 장소 골라냄

In [11]:
seoul_salon = pd.read_csv(data_path+"seoul_salon.csv", encoding=encoding)
seoul_salon = seoul_salon.loc[seoul_salon["상세영업상태명"]=="영업", :]
processed_salon = pd.DataFrame()
gu_code=[3100000, 3090000, 3060000, 3070000]
for i in gu_code:
    processed_salon = pd.concat([ processed_salon, seoul_salon.loc[seoul_salon["개방자치단체코드"] == i, ["지번주소", "도로명주소"]] ])
processed_salon = processed_salon.reset_index(drop=True) 
processed_salon

,지번주소,도로명주소
0,서울특별시 노원구 상계동 706-1 2층,"서울특별시 노원구 노해로75길 20-21 (상계동,2층)"
1,서울특별시 노원구 상계동 710 문화빌딩 지하1층,"서울특별시 노원구 노해로77길 14-3, 지하1층 (상계동, 문화빌딩)"
2,서울특별시 노원구 상계동 730-5 지하2호,"서울특별시 노원구 동일로 1393, 지하층 지하2호 (상계동)"
3,서울특별시 노원구 상계동 711-9 노블레스관광호텔 3층,"서울특별시 노원구 노해로77길 22, 노블레스관광호텔 3층 (상계동)"
4,서울특별시 노원구 상계동 734-4번지 동방빌딩,"서울특별시 노원구 노해로 456, 동방빌딩 지하층 (상계동)"
...,...,...
60,서울특별시 성북구 동선동4가 7 외 1필지,"서울특별시 성북구 아리랑로 4 (동선동4가,외 1필지)"
61,서울특별시 성북구 동소문동5가 85번지,서울특별시 성북구 동소문로20길 12 (동소문동5가)
62,서울특별시 성북구 동선동1가 2-12번지,서울특별시 성북구 동소문로20나길 3 (동선동1가)
63,서울특별시 성북구 보문동4가 77-2번지 브라운관광호텔 지하1층,"서울특별시 성북구 보문로 78 (보문동4가,브라운관광호텔 지하1층)"


In [12]:
processed_salon.to_csv(data_path+"processed_salon.csv", index=False,  encoding=encoding)